In [1]:
import os
os.chdir("/Users/laurensuarez/Desktop/deep_wisdom/deep_wisdom_django/dwsite")
from DeepWisdom import *
#os.chdir("/Users/laurensuarez/Desktop/deep_wisdom/deep_wisdom_django/dwsite/data")
os.getcwd()

/Users/laurensuarez/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'/Users/laurensuarez/Desktop/deep_wisdom/deep_wisdom_django/dwsite'

In [2]:
dw = DeepWisdom()
results_dict=dw.query("love", K=100)

Constructing DeepWisdom
Loading data.
.
.
Loading model weights.
X Shape:  (31102, 12302)
y Shape:  (31102, 31102)
Load complete.
.
Load complete.
Connecting to database.
Connecting to database.


In [3]:
import requests
import re
import string
from bs4 import BeautifulSoup
import bs4
import pickle
import numpy as np
import keras
import nltk
import string
import pickle
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Flatten
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size

def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

### GET DATA
with open("data/bible_data_20181129_update.pkl", "rb") as handle:
    bible_data=pickle.load(handle)

In [4]:
#int2verse=pickle_load("int2verse_mapping.pkl")
#verse2int=pickle_load("verse2int_mapping.pkl")
kjv_bible_mapping=pickle_load("data/kjv_bible_mapping.pkl")
print("Successful loading.")


Successful loading.


In [65]:
import re
import sys
from collections import defaultdict
from __future__ import division
import random
import nltk
import string

class ChatBot:
    
    """
    Members we expect PeterChatBot to have after proper construction.
    """
    bible_data=None
    pattern=None
    corpus=None
    corpusClean=None
    word2idx = None
    idx2word = None
    vocab_size = None
    bigram_counts = None
    bigram_possibilities = None
    P = None
    
    options={'Prophets':r"""(Isaiah|Jeremiah|Lamentations|
                              Ezekiel|Daniel|Hosea|Joel|Amos|Obadiah|Jonah|Micah|
                              Micah|Nahum|Habakkuk|Zephaniah|Haggai|Zechariah|
                              Malachi)""",
             'Apostles':r"""(Romans|.*Corinthians.*|Galatians|Philippians|
                              Ephesians|Colossians|.*Thessalonians.*|Hebrews|
                              .*Timothy.*|Titus|Philemon|
                              .*John.*|Revelation|
                              .*Peter.*|James)""",
             'Old':r"""(Genesis|Exodus|Leviticus|Numbers|Deuteronomy|
                              Joshua|Judges|Ruth|
                              .*Saumuel.*|.*Kings.*|.*Chronicles.*|
                              Ezra|Nehemiah|Esther|.*Psalm.*|.*Proverb.*|
                              Ecclesiastes|.*Song.*|Isaiah|Jeremiah|Lamentations|
                              Ezekiel|Daniel|Hosea|Joel|Amos|Obadiah|Jonah|Micah|
                              Micah|Nahum|Habakkuk|Zephaniah|Haggai|Zechariah|
                              Malachi)""",
             'New':r"""(Matthew|Mark|Luke|Acts|
                              Romans|.*Corinthians.*|Galatians|Philippians|
                              Ephesians|Colossians|.*Thessalonians.*|Hebrews|
                              .*Timothy.*|Titus|Philemon|.*John.*|Revelation|
                              .*Peter.*|Jude|James
                                )""",
             'Paul':r"(Romans|.*Corinthians.*|Galatians|Philippians|Ephesians|Colossians|.*Thessalonians.*|Hebrews|.*Timothy.*|Titus|Philemon)",
             'John':r"(.*John.*|Revelation)",
             'James':r"(James)",
             'Peter':r"(.*Peter.*)",
             'Jesus':r"(Matthew|Mark|Luke|John)",
             'Moses':r"(Genesis|Exodus|Leviticus|Numbers|Deuteronomy)",
             'Joshua':r"(Joshua)",
             'Job':r"(Job)",
             'David':r"(.*Samuel.*|.*Psalm.*)",
             'Solomon':r"(.*Proverb.*|.*Ecclesiastes.*|.*Song.*)",
             'Daniel':r"(Daniel)",
             'Isaiah':r"(Isaiah)",
             'Jeremiah':r"(Jeremiah|Lamentations)",
             'Ezekiel':r"(Ezekiel)",
             'Ruth':r"(Ruth)",
             'Nehemiah':r"(Nehemiah)",
             'Esther':r"(Esther)",
             'Hosea':r"(Hosea)",
             'Joel':r"(Joel)",
             'Jonah':r"(Jonah)",
             'Micah':r"(Micah)",
             'Nahum':r"(Nahum)",
             'Habakkuk':r"(Habakkuk)",
             'Zephaniah':r"(Zephaniah)",
             'Haggai':r"(Haggai)",
             'Zechariah':r"(Zechariah)",
             'Malachi':r"(Malachi)"
            }
    
    
    def __init__(self, who, verbose=True):
        print("Building {} Chat Bot".format(who))
        assert(who in list(self.options.keys()), "{} is not a valid character, try someone else.".format(who))
        self.who=who
        
    def getCorpus(self, pattern, dw_bible_mapping):
        
        """
        pattern should be a regular expression that will parse down any books containing
        some pattern of interest. For Peter this should just be (.*Peter.*).
        """
        self.corpus=[]
        
        # Get corpus of hits within this character's pattern
        for i in list(dw_bible_mapping.keys()):
            result=re.match(pattern,i)
            if not result is None:
                #print(result)
                self.corpus.append(dw_bible_mapping[i])
        
        self.corpusClean=[]
        for i in self.corpus:
            newVerse=[]
            for j in nltk.word_tokenize(i):
                newVerse.append(j.lower())
            self.corpusClean.append(newVerse)
        return self.corpusClean
    
    def getLookupTables(self, corpusClean):
        w2i={}
        i2w={}
        allWords = []
        for i in corpusClean:
            allWords = allWords + i
        for i,word in enumerate(allWords, 1):
            #print(i)
            w2i[word]=i
            i2w[i]=word
        self.word2idx={'UNK':0}
        self.idx2word={0:'UNK'}
        counter=1
        for k,v in w2i.items():
            self.word2idx[k]=counter
            self.idx2word[counter]=k
            counter+=1
        self.vocab_size=len(list(self.word2idx.items()))
        return self.word2idx, self.idx2word, self.vocab_size
    
    def getBigrams(self, corpusClean):
        """
        Usage:
            @param: sentences = ["this is a sentence1", "this is a sentence2", ..."this is a sentenceN"]
        """
        self.bigram_counts={}
        for sentence in self.corpusClean:
            N=len(sentence)
            for j in range(0,N-1):
                k=(sentence[j], sentence[j+1])
                try:
                    self.bigram_counts[k]+=1
                except Exception as e:
                    self.bigram_counts[k]=1
        return self.bigram_counts
    
    def getBigramPossibilities(self, bigram_counts, corpusClean):
        # Get list of bigram possibilities
        self.bigrams_possibilities=defaultdict(dict) #{k:[(w2j:count), w2, ..., wN]}
        found=set()
        for sentence in corpusClean:
            N=len(sentence)
            for j in range(0,N-1):
                # Get bigram lists
                w1=sentence[j]
                w2=sentence[j+1]
                bg_count = bigram_counts[(w1,w2)]    
                if bg_count > 0:
                    self.bigrams_possibilities[w1][w2] = bg_count
                else:
                    print("Skipping {} {}".format(w1, w2))
        return self.bigrams_possibilities
    
    def getMarkovMatrix(self, bigram_possibilities, vocab_size, w2i, i2w, smooth=0.01):
        self.P = np.ones((vocab_size,vocab_size), dtype=np.float)*smooth
        #print(self.P.shape)
        for w1,v in bigram_possibilities.items():
            for w2, c in v.items():
                #print("{}({}) {}({})".format(w1, word2idx[w1], w2, word2idx[w2]))
                self.P[w2i[w1],w2i[w2]]+=c
                #if w1 == 'suffering' or w2 == 'suffering':
                #    print(P[word2idx[w1],word2idx[w2]])
        self.P = self.P**1.75
        self.P /= self.P.sum(axis=1, keepdims=True)

        return self.P
    
    # INTERFACE WITH BOT
    def getNextWord(self, word, perfect=False, pure_chance=False, perfect_threshold=0.5):
        #np.random.choice(numpy.arange(1, 7), p=[0.1, 0.05, 0.05, 0.2, 0.4, 0.2])
        assert(not self.word2idx is None, "Please construct Peter Chat Bot first.")
        assert(not self.idx2word is None, "Please construct Peter Chat Bot first.")
        assert(not self.P is None, "Please construct Peter Chat Bot first.")

        try:
            i=self.word2idx[word]
        except:
            i=self.word2idx['UNK']
        #print(i)
        if perfect:
            j=self.P[i,:].argmax()
            # If UNK token, just choose randomly.
            if j == 0:
                j=np.random.choice(np.arange(0, self.P.shape[1]), p=list(self.P[i,:]))
        else:
            if pure_chance:
                j=np.random.choice(np.arange(0, self.P.shape[1]), p=list(self.P[i,:]))
            else:
                # 50/50 explore exploit
                if random.uniform(0, 1) < perfect_threshold:
                    j=np.random.choice(np.arange(0, self.P.shape[1]), p=list(self.P[i,:]))
                else:
                    j=self.P[i,:].argmax()
        word=self.idx2word[j]
        return word

    # HELPER FUNCTION FOR INTERFACE
    def makeSentenceFromJumble(self, sent):
        if not isinstance(sent, str):
            sent = " ".join([i for i in sent])
        w0=""
        for word in nltk.sent_tokenize(sent):
            for w in nltk.word_tokenize(word):
                if w in string.punctuation:
                    w0+=w
                else:
                    w0+=" "
                    w0+=w
        return w0
    
    # INTERFACE WITH USER
    def askTopic(self, word, results, sequenceLength=50, perfectFitLength=5, perfectThreshold=0.5, verbose=False):
        
        """
        Idea ...
        1. Get topic results from DW
        2. Sort through just pauls take on this topic
        3. Construct MC from this
        4. Let paul start talking from within this chain
        """
        #self.dw_bible_mapping=dw.query(word, K=100)
        self.dw_bible_mapping=results
        self.pattern=self.options[self.who]
        self.corpusClean = self.getCorpus(self.pattern, self.dw_bible_mapping)
        
        # Determine if this speaker is proficient on the topic
        if len(self.corpusClean) < 1:
            if verbose:
                print("Speaker: {}\nTopic: {}\nCorpus Size: {}".format(self.who, word, len(self.corpusClean)))
                print("{} NOT PROFICIENT ENOUGH ON {} TO EXPOUND.".format(self.who, word))
            return self.who, len(self.corpusClean), None

        
        # Otherwise build Markov Model for this speaker on the relevant texts returned from deepwisdom
        self.word2idx, self.idx2word, self.vocab_size = self.getLookupTables(self.corpusClean)
        self.bigram_counts = self.getBigrams(self.corpusClean)
        self.bigrams_possibilities = self.getBigramPossibilities(self.bigram_counts, self.corpusClean)
        self.P = self.getMarkovMatrix(self.bigrams_possibilities, self.vocab_size, self.word2idx, self.idx2word, smooth=0.01)
        
        if verbose:
            print("Speaker: {}\nTopic: {}\nCorpus Size: {}".format(self.who, word, len(self.corpusClean)))
        sentence=[]
        # Start off perfectly
        for _ in range(perfectFitLength):
            sentence.append(word)
            word=self.getNextWord(word, perfect=True, perfect_threshold=perfectThreshold)
            
                
        # Continue probabilistically
        for _ in range(sequenceLength):
            sentence.append(word)
            word=self.getNextWord(word, pure_chance=True)
        
        if verbose:
            for w in self.makeSentenceFromJumble(sentence):
                sys.stdout.write(w)
            print("\n")
        return self.who, len(self.corpusClean), self.makeSentenceFromJumble(sentence)

def test():
    bots=[]
    #s=set(["Old", "New"])
    keys=set(list(ChatBot.options.keys()))
    #d=keys.difference(s)

    for k in keys:
        cb=ChatBot(who=k)
        bots.append(cb)
    topics=['hope', 'life', 'sheep', 'resurrection']
    for t in topics:
        #print("Topic {}".format(t))
        results=dw.query(t, K=50)
        for b in bots:
            who, size, sent = b.askTopic(t, results)
            if sent is None:
                continue
            print("{}-{}-{}".format(who, size, sent))
        input("Ready for next topic?")        
test()


<ipython-input-65-b3293564e30c>:80: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(who in list(self.options.keys()), "{} is not a valid character, try someone else.".format(who))
<ipython-input-65-b3293564e30c>:176: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(not self.word2idx is None, "Please construct Peter Chat Bot first.")
<ipython-input-65-b3293564e30c>:177: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(not self.idx2word is None, "Please construct Peter Chat Bot first.")
<ipython-input-65-b3293564e30c>:178: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(not self.P is None, "Please construct Peter Chat Bot first.")


Building Prophets Chat Bot
Building Moses Chat Bot
Building Isaiah Chat Bot
Building Joshua Chat Bot
Building Hosea Chat Bot
Building Zephaniah Chat Bot
Building Solomon Chat Bot
Building Malachi Chat Bot
Building Zechariah Chat Bot
Building Daniel Chat Bot
Building Paul Chat Bot
Building Old Chat Bot
Building Peter Chat Bot
Building James Chat Bot
Building Esther Chat Bot
Building Jonah Chat Bot
Building Nahum Chat Bot
Building Haggai Chat Bot
Building Job Chat Bot
Building Ruth Chat Bot
Building Nehemiah Chat Bot
Building New Chat Bot
Building Jeremiah Chat Bot
Building Ezekiel Chat Bot
Building Habakkuk Chat Bot
Building Jesus Chat Bot
Building David Chat Bot
Building Apostles Chat Bot
Building Micah Chat Bot
Building John Chat Bot
Building Joel Chat Bot
Connecting to database.
Connecting to database.
Prophets-5- hope: even to the poor and thine ears shall increase their joy in your heart. thine ears shall rejoice in it was well with him: was not the holy one of hope: was well with 

KeyboardInterrupt: 